# Diff in Diff using Python

https://medium.com/@sadhaverajasekar/diff-in-diff-testing-python-f24835330bc8

Let’s say you have launched an offer (50% off on all products) during Thanksgiving and the sales of the product are at an all-time high. Great, the offer worked! But wait a minute, you notice that historically the product sales always peaks during Thanksgiving. In this case, you cannot randomly allocate a control that exactly matches your target performance before the launch of the offer. So how do you measure the actual impact of the offer? DID testing is the answer!

## Terms
- Target: Segment of customers who are exposed to the treatment (say, certain zip codes targeted for the 50% offer)
- Control or Holdout: Segment of customers who are not exposed to the treatment (say, zip codes that are similar in performance to the target before offer launch)
- Treatment: New method that is intended to be tested (50% offer)

Note: Since the target and control population are chosen according to the zip code and not designed individually for customers, there will inherent differences that exist even before the offer is launched

Data link - http://dss.princeton.edu/training/Panel101.dta - converted to csv

Stata version of the codes below - http://www.princeton.edu/~otorres/DID101.pdf

Treatment introducted in 1994 to countries E, F, G. Control countries - A, B, C, D

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [10]:
#df = pd.read_stata('Panel101.dta')
#df.to_csv('panel101.csv', index=False)

df = pd.read_csv('panel101.csv')

In [11]:
df.head()

,country,year,y,y_bin,x1,x2,x3,opinion,op
0,A,1990,1.342788e+09,1.0,0.277904,-1.107956,0.282554,Str agree,1.0
1,A,1991,-1.899661e+09,0.0,0.320685,-0.948720,0.492538,Disag,0.0
2,A,1992,-1.123436e+07,0.0,0.363466,-0.789484,0.702523,Disag,0.0
3,A,1993,2.645775e+09,1.0,0.246144,-0.885533,-0.094391,Disag,0.0
4,A,1994,3.008335e+09,1.0,0.424623,-0.729768,0.946131,Disag,0.0


In [14]:
# Adding time variable, treatment occured in 1994 ; segmenting anything before 1994 as pre-period (0) 
df['time'] = np.where(df['year'] >= 1994, 1, 0 )

In [16]:
df.head()

,country,year,y,y_bin,x1,x2,x3,opinion,op,time
0,A,1990,1.342788e+09,1.0,0.277904,-1.107956,0.282554,Str agree,1.0,0
1,A,1991,-1.899661e+09,0.0,0.320685,-0.948720,0.492538,Disag,0.0,0
2,A,1992,-1.123436e+07,0.0,0.363466,-0.789484,0.702523,Disag,0.0,0
3,A,1993,2.645775e+09,1.0,0.246144,-0.885533,-0.094391,Disag,0.0,0
4,A,1994,3.008335e+09,1.0,0.424623,-0.729768,0.946131,Disag,0.0,1


In [18]:
df['country'].unique()

array(['A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype=object)

In [23]:
#creating a treated variable to identify target (1) and the holdout (0) groups
df['treated'] = np.where(df['country'] >= 'E' , 1,0 )
df.head()

,country,year,y,y_bin,x1,x2,x3,opinion,op,time,treated,did
0,A,1990,1.342788e+09,1.0,0.277904,-1.107956,0.282554,Str agree,1.0,0,0,0
1,A,1991,-1.899661e+09,0.0,0.320685,-0.948720,0.492538,Disag,0.0,0,0,0
2,A,1992,-1.123436e+07,0.0,0.363466,-0.789484,0.702523,Disag,0.0,0,0,0
3,A,1993,2.645775e+09,1.0,0.246144,-0.885533,-0.094391,Disag,0.0,0,0,0
4,A,1994,3.008335e+09,1.0,0.424623,-0.729768,0.946131,Disag,0.0,1,0,0


In [24]:
#creating interaction variable
df['did'] = df['time'] * df['treated']
df.tail()

,country,year,y,y_bin,x1,x2,x3,opinion,op,time,treated,did
65,G,1995,1.323696e+09,1.0,1.087187,-1.409817,2.829808,Str disag,0.0,1,1,1
66,G,1996,2.545242e+08,1.0,0.781076,-1.328000,4.278224,Str agree,1.0,1,1,1
67,G,1997,3.297033e+09,1.0,1.257880,-1.577367,4.587326,Disag,0.0,1,1,1
68,G,1998,3.011821e+09,1.0,1.242777,-1.601218,6.113762,Disag,0.0,1,1,1
69,G,1999,3.296283e+09,1.0,1.234200,-1.621761,7.168922,Disag,0.0,1,1,1


In [36]:
# plitting variables into dependent (x) and independent (y) variables
#x = df.iloc[:, 9:12]
#y = df.iloc[:, 2]
x = df[['time', 'treated', 'did']]
y = df['y']

In [37]:
x.head()

,time,treated,did
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,1,0,0


In [38]:
y.head()

0    1.342788e+09
1   -1.899661e+09
2   -1.123436e+07
3    2.645775e+09
4    3.008335e+09
Name: y, dtype: float64

In [39]:
model = LinearRegression().fit(x, y)

In [40]:
model.coef_

array([ 2.28945465e+09,  1.77596967e+09, -2.51951163e+09])

In [41]:
# Model to get the summary statistics
import statsmodels.api as sm

In [42]:
X2 = sm.add_constant(x)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     1.984
Date:                Wed, 08 Dec 2021   Prob (F-statistic):              0.125
Time:                        20:21:27   Log-Likelihood:                -1623.7
No. Observations:                  70   AIC:                             3255.
Df Residuals:                      66   BIC:                             3264.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.581e+08   7.38e+08      0.485      0.6

/Users/harrywang/sandbox/tutorial-buffet/diff-in-diff/venv/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Coefficients of the variable did provides the true lift of the treatment. In this case, the treatment has a negative performance on the outcome variable.